In [18]:
VIDEO_PATH = "./resources/kemu3.mp4"
IMAGES__INITIAL_DIR = "./images/initial/"

import os
import cv2
import shutil

video = cv2.VideoCapture(VIDEO_PATH)

fps = video.get(cv2.CAP_PROP_FPS)
fps_count = video.get(cv2.CAP_PROP_FRAME_COUNT)

print("The total frame rate of the video is " + str(fps_count) + " frames and the frame rate is " + str(fps) + " frames.")

# 判断images_initial目录是否存在
isExist = os.path.exists(IMAGES__INITIAL_DIR)
if isExist:
    shutil.rmtree(IMAGES__INITIAL_DIR)
os.mkdir(IMAGES__INITIAL_DIR, mode=0o777)

index = 0
if video.isOpened():
    rval, frame = video.read()
else:
    rval = False
    print("Error opening video stream or file.")
    
while rval:
    rval, frame = video.read()
    if frame is None:
        break
    else:
        cv2.imwrite(IMAGES__INITIAL_DIR + str(index) + ".png", frame)
    index += 1
    print("["+ str(int(100 * index / (fps_count - 1))) +"%] " + "Split ", str(index), " screenshots successfully.", end="\r")

The total frame rate of the video is 1433.0 frames and the frame rate is 60.0 frames.


In [21]:
IMAGES__INITIAL_DIR = './images/initial/'
IMAGES__REMBG_DIR = './images/rembg/'

import os
import sys
import rembg

# 判断images_initial目录是否存在
isExist = os.path.exists(IMAGES__INITIAL_DIR)
if not isExist:
    print("Please take a screenshot of the video in different frames.")
    sys.exit(0)

# 判断images_rembg目录是否存在
isExist = os.path.exists(IMAGES__REMBG_DIR)
if isExist:
    shutil.rmtree(IMAGES__REMBG_DIR)
os.mkdir(IMAGES__REMBG_DIR, mode=0o777)

images = os.listdir(IMAGES__INITIAL_DIR)
images.sort(key=lambda x: int(x.split(".")[0]))

length = len(images)

index = 0
for image in images:
    full_path = os.path.join(IMAGES__INITIAL_DIR, image)
    save_path = os.path.join(IMAGES__REMBG_DIR, image)
    with open(full_path, "rb") as i:
        with open(save_path, "wb") as o:
            ipt = i.read()
            opt = rembg.remove(ipt)
            o.write(opt)
            index += 1
            print("[" + str(int(index * 100 / length)) + "%] Removed the background of " + str(index) + "images successfully.", end="\r")

In [38]:
IMAGES__REMBG_DIR = "./images/rembg/"
IMAGES__TWO_DIR = "./images/two/"

import os
import sys
import cv2
import numpy as np

# 判断images_rembg目录是否存在
isExist = os.path.exists(IMAGES__REMBG_DIR)
if not isExist:
    print("Please remove the background firstly.")
    sys.exit(0)
    
# 判断images_rembg目录是否存在
isExist = os.path.exists(IMAGES__TWO_DIR)
if isExist:
    shutil.rmtree(IMAGES__TWO_DIR)
os.mkdir(IMAGES__TWO_DIR, mode=0o777)

images = os.listdir(IMAGES__REMBG_DIR)
images.sort(key=lambda x: int(x.split(".")[0]))

length = len(images)

index = 0
for image in images:
    full_path = os.path.join(IMAGES__REMBG_DIR, image)
    save_path = os.path.join(IMAGES__TWO_DIR, image)
    matrix = cv2.imread(full_path, cv2.IMREAD_UNCHANGED)
    row, col, channel = matrix.shape
    alpha = np.zeros((row, col), dtype=np.uint8)
    for r in range(row):
        for c in range(col):
            alpha[r, c] = matrix[r, c, channel - 1]
    index += 1
    cv2.imwrite(IMAGES__TWO_DIR + str(index) + ".png", alpha)
    print("[" + str(int(index * 100 / length)) + "%] " + "The alpha channels of " + str(index) + " images were successfully extracted.", end="\r")

In [62]:
IMAGES__TWO_DIR = "./images/two/"
NPY_PATH = "./data.npy"

import os
import cv2
import sys
import numpy as np

# 判断images_two目录是否存在
isExist = os.path.exists(IMAGES__TWO_DIR)
if not isExist:
    print("Please binarize the image firstly.")
    sys.exit(0)

images = os.listdir(IMAGES__TWO_DIR)
images.sort(key=lambda x: int(x.split(".")[0]))

length = len(images)

index = 0
__list = []
save_path = os.path.join(NPY_PATH)
for image in images:
    full_path = os.path.join(IMAGES__TWO_DIR, image)
    matrix = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)
    __list.append(matrix)
    index += 1
    print("[" + str(int(index * 100 / length)) + "%] The matrix of " + str(index) + " images was successfully extracted.", end="\r")
npy = np.array(__list)
np.save(save_path, npy)

In [22]:
NPY_PATH = "./data.npy"
SAVE_PATH = "./data_zipped.npy"

import numpy as np

data = np.load(NPY_PATH)
total, row, col = data.shape

zipped = []
for i in range(total):
    tmp = np.zeros((int(row / 4), int(col / 4)), dtype=np.uint8)
    r = 0
    while r < row and r + 3 < row:
        c = 0
        while c < col and c + 3 < col:
            tmp[int(r / 4), int(c / 4)] = int(sum([
                data[i, r, c + 0], 
                data[i, r, c + 1], 
                data[i, r, c + 2], 
                data[i, r, c + 3], 
                data[i, r + 1, c + 0], 
                data[i, r + 1, c + 1], 
                data[i, r + 1, c + 2], 
                data[i, r + 1, c + 3], 
                data[i, r + 2, c + 0], 
                data[i, r + 2, c + 1], 
                data[i, r + 2, c + 2], 
                data[i, r + 2, c + 3], 
                data[i, r + 3, c + 0], 
                data[i, r + 3, c + 1], 
                data[i, r + 3, c + 2], 
                data[i, r + 3, c + 3], 
            ]) / 16)
            c += 4
        r += 4
    zipped.append(tmp)
    print("[" + str(int(i * 100 / total)) + "%] The matrix of " + str(i) + " images was zipped successfully.", end="\r")

zipped = np.array(zipped)

np.save(SAVE_PATH, zipped)

In [18]:
NPY_PATH = "./data_zipped.npy"
SAVE_PATH = "./data_zipped.npy"

import math
import numpy as np

data = np.load(NPY_PATH)
total, row, col = data.shape

zipped = []
for i in range(total):
    tmp = np.zeros((math.floor(row / 4), math.floor(col / 4)), dtype=np.uint8)
    r = 0
    while r < row and r + 3 < row:
        c = 0
        while c < col and c + 3 < col:
            tmp[math.floor(r / 4), math.floor(c / 4)] = int(sum([
                data[i, r, c + 0], 
                data[i, r, c + 1], 
                data[i, r, c + 2], 
                data[i, r, c + 3], 
                data[i, r + 1, c + 0], 
                data[i, r + 1, c + 1], 
                data[i, r + 1, c + 2], 
                data[i, r + 1, c + 3], 
                data[i, r + 2, c + 0], 
                data[i, r + 2, c + 1], 
                data[i, r + 2, c + 2], 
                data[i, r + 2, c + 3], 
                data[i, r + 3, c + 0], 
                data[i, r + 3, c + 1], 
                data[i, r + 3, c + 2], 
                data[i, r + 3, c + 3], 
            ]) / 16)
            c += 4
        r += 4
    zipped.append(tmp)
    print("[" + str(int(i * 100 / total)) + "%] The matrix of " + str(i) + " images was zipped successfully.", end="\r")

zipped = np.array(zipped)

np.save(SAVE_PATH, zipped)

In [20]:
NPY_PATH = "./data_zipped.npy"
SAVE_PATH = "./data_zipped.npy"

import math
import numpy as np

data = np.load(NPY_PATH)
total, row, col = data.shape

zipped = []
for i in range(total):
    tmp = np.zeros((math.floor(row / 4), math.floor(col / 4)), dtype=np.uint8)
    r = 0
    while r < row and r + 3 < row:
        c = 0
        while c < col and c + 3 < col:
            tmp[math.floor(r / 4), math.floor(c / 4)] = int(sum([
                data[i, r, c + 0], 
                data[i, r, c + 1], 
                data[i, r, c + 2], 
                data[i, r, c + 3], 
                data[i, r + 1, c + 0], 
                data[i, r + 1, c + 1], 
                data[i, r + 1, c + 2], 
                data[i, r + 1, c + 3], 
                data[i, r + 2, c + 0], 
                data[i, r + 2, c + 1], 
                data[i, r + 2, c + 2], 
                data[i, r + 2, c + 3], 
                data[i, r + 3, c + 0], 
                data[i, r + 3, c + 1], 
                data[i, r + 3, c + 2], 
                data[i, r + 3, c + 3], 
            ]) / 16)
            c += 4
        r += 4
    zipped.append(tmp)
    print("[" + str(int(i * 100 / total)) + "%] The matrix of " + str(i) + " images was zipped successfully.", end="\r")

zipped = np.array(zipped)

np.save(SAVE_PATH, zipped)

In [21]:
NPY_PATH = "./data_zipped.npy"
SAVE_PATH = "./data_zipped.npy"

import math
import numpy as np

data = np.load(NPY_PATH)
total, row, col = data.shape

zipped = []
for i in range(total):
    tmp = np.zeros((math.floor(row / 4), math.floor(col / 4)), dtype=np.uint8)
    r = 0
    while r < row and r + 3 < row:
        c = 0
        while c < col and c + 3 < col:
            tmp[math.floor(r / 4), math.floor(c / 4)] = int(sum([
                data[i, r, c + 0], 
                data[i, r, c + 1], 
                data[i, r, c + 2], 
                data[i, r, c + 3], 
                data[i, r + 1, c + 0], 
                data[i, r + 1, c + 1], 
                data[i, r + 1, c + 2], 
                data[i, r + 1, c + 3], 
                data[i, r + 2, c + 0], 
                data[i, r + 2, c + 1], 
                data[i, r + 2, c + 2], 
                data[i, r + 2, c + 3], 
                data[i, r + 3, c + 0], 
                data[i, r + 3, c + 1], 
                data[i, r + 3, c + 2], 
                data[i, r + 3, c + 3], 
            ]) / 16)
            c += 4
        r += 4
    zipped.append(tmp)
    print("[" + str(int(i * 100 / total)) + "%] The matrix of " + str(i) + " images was zipped successfully.", end="\r")

zipped = np.array(zipped)

np.save(SAVE_PATH, zipped)

In [19]:
NPY__ZIPPED_PATH = "./data_zipped.npy"

import numpy as np
from PIL import Image

data = np.load(NPY__ZIPPED_PATH)

image = Image.fromarray(data[1000])
image.show()